In [ ]:
%%HTML
<style>
.container { width: 100% }
</style>

# Checking the Equivalence of Regular Expressions

In [ ]:
%run Regexp-2-NFA.ipynb
%run NFA-2-DFA.ipynb

Given two sets `A` and `B`, the function `cartesian_product(A, B)` computes the 
<em style="color:blue">cartesian product</em> $A \times B$ which is defined as
$$ A \times B := \{ (x, y) \mid x \in A \wedge y \in B \}. $$

In [ ]:
def cartesian_product(A, B):
    return { (x, y) for x in A
                    for y in B
           }

In [ ]:
cartesian_product({1, 2}, {'a', 'b'})

Given to deterministic <span style="font-variant:small-caps;">Fsm</span>s `F1` and `F2`,
the expression `fsm_complement(F1, F2)` computes a deterministic 
<span style="font-variant:small-caps;">Fsm</span> that recognizes the language  $L(F_1)\backslash L(F_2)$.

In [ ]:
def fsm_complement(F1, F2):
    States1, Sigma, delta1, q1, A1 = F1
    States2,     _, delta2, q2, A2 = F2
    States = cartesian_product(States1, States2)
    delta  = {}
    for (p1, p2) in States:
        for c in Sigma:
            delta[(p1, p2), c] = (delta1[p1, c], delta2[p2, c])
    return (States, Sigma, delta, (q1, q2), cartesian_product(A1, States2 - A2))

In [ ]:
def regexp2DFA(r, Sigma):
    converter = RegExp2NFA(Sigma)
    States, Sigma, delta, q0, A = converter.toNFA(r)
    return nfa2dfa((States, Sigma, delta, q0, A))

In [ ]:
def is_empty(F):
    States, Sigma, delta, q0, Accepting = F
    Reachable = { q0 }
    NewFound  = { q0 }
    while True:
        NewFound = { delta[q, c] for q in NewFound
                                 for c in Sigma 
                   }
        if NewFound <= Reachable:
            break
        Reachable |= NewFound
    return Reachable & Accepting == set()

In [ ]:
%run FSM-2-Dot.ipynb

In [ ]:
def regExpEquiv(r1, r2, Sigma):
    F1 = regexp2DFA(r1, Sigma)
    F2 = regexp2DFA(r2, Sigma)    
    r1_minus_r2 = fsm_complement(F1, F2)
    r2_minus_r1 = fsm_complement(F2, F1)
    return is_empty(r1_minus_r2) and is_empty(r2_minus_r1)